In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.utils import to_categorical

import copy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
import random
random.seed()

import tensorflow.python.keras
from tensorflow.python.keras.models import *
from tensorflow.python.keras.layers import *

### There are many columns, but many represent the same features (e.g. monthly installment & total loan amount). So I chose the distinct features among the dataset.

In [2]:
goodcol = ['term','installment','grade','sub_grade','home_ownership','annual_inc','verification_status','loan_status','purpose','addr_state','dti','delinq_2yrs','revol_util']
cr_data = pd.read_csv("loan.csv", skipinitialspace=True, usecols=goodcol)
cr_data.head()

,term,installment,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,revol_util
0,36 months,84.92,C,C1,RENT,55000.0,Not Verified,Current,debt_consolidation,NY,18.24,0.0,10.3
1,60 months,777.23,D,D2,MORTGAGE,90000.0,Source Verified,Current,debt_consolidation,LA,26.52,0.0,24.2
2,36 months,180.69,D,D1,MORTGAGE,59280.0,Source Verified,Current,debt_consolidation,MI,10.51,0.0,19.1
3,36 months,146.51,D,D2,MORTGAGE,92000.0,Source Verified,Current,debt_consolidation,WA,16.74,0.0,78.1
4,60 months,731.78,C,C4,MORTGAGE,57250.0,Not Verified,Current,debt_consolidation,MD,26.35,0.0,3.6


### For the target, "loan status", loans that are still ongoing are dropped. Also, loans that are late or called off are categorized as "default".

In [3]:
def clean_data(df):
    df = df[df.home_ownership != 'ANY']
    df = df[df.home_ownership != 'OTHER']
    df = df[df.home_ownership != 'NONE']
    df = df[df.loan_status != 'Current']
    df = df[df.loan_status != 'In Grace Period']
    df = df.replace("Charged Off", "Default")
    df = df.replace("Late (31-120 days)", "Default")
    df = df.replace("Late (16-30 days)", "Default")
    df = df.replace("Does not meet the credit policy. Status:Fully Paid", "Fully Paid")
    df = df.replace("Does not meet the credit policy. Status:Charged Off", "Default")
    return df

### Even after dropping the incomplete rows and current on-going loans, we still have 1.3 million data points

In [4]:
cr_data.shape

(2260668, 13)

In [5]:
cr_data = cr_data.dropna()
cr_data = clean_data(cr_data)
cr_data = cr_data.reset_index(drop=True)
cr_data.shape

(1330271, 13)

In [6]:
cr_data.head()

,term,installment,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,revol_util
0,36 months,1151.16,D,D5,MORTGAGE,100000.0,Source Verified,Fully Paid,debt_consolidation,CA,30.46,0.0,37.0
1,60 months,975.71,C,C4,MORTGAGE,45000.0,Verified,Fully Paid,credit_card,OH,50.53,0.0,64.5
2,36 months,622.68,A,A3,MORTGAGE,100000.0,Not Verified,Fully Paid,credit_card,WA,18.92,0.0,29.9
3,36 months,147.99,B,B3,RENT,38500.0,Not Verified,Fully Paid,credit_card,TX,4.64,0.0,15.3
4,36 months,345.18,E,E5,MORTGAGE,450000.0,Verified,Fully Paid,credit_card,MA,12.37,0.0,65.7


### The current precision of the bank's exisitng model is about 78.38%

In [7]:
print(cr_data['loan_status'].value_counts()[0]/len(cr_data))

0.7837628573426016


## Model 1: Discrete Graphical Model

All non-categorical data is converted into categories. Due to limited computing power, only a small subset (20,000) of the samples are "tested". The value of P(loan status|annual income, installment, revol_util, delinq_2yrs) is computed. In other word, a huge table of probabilities is generated. 

In [8]:
def to_discrete(df):
    df.loc[(df['installment'] <= 200), ['installment']] = 0
    df.loc[(df['installment'] > 200) & (df['installment'] <= 400), ['installment']] = 1
    df.loc[(df['installment'] > 400) & (df['installment'] <= 600), ['installment']] = 2
    df.loc[(df['installment'] > 600) & (df['installment'] <= 800), ['installment']] = 3
    df.loc[(df['installment'] > 800), ['installment']] = 4
    df.loc[df['installment'] == 0, ['installment']] = '0~200'
    df.loc[df['installment'] == 1, ['installment']] = '200~400'
    df.loc[df['installment'] == 2, ['installment']] = '400~600'
    df.loc[df['installment'] == 3, ['installment']] = '600~800'
    df.loc[df['installment'] == 4, ['installment']] = '800+'
    
    df.loc[(df['annual_inc'] > 0) & (df['annual_inc'] <= 40000), ['annual_inc']] = 0
    df.loc[(df['annual_inc'] > 40000) & (df['annual_inc'] <= 60000), ['annual_inc']] = 40
    df.loc[(df['annual_inc'] > 60000) & (df['annual_inc'] <= 80000), ['annual_inc']] = 60
    df.loc[(df['annual_inc'] > 80000) & (df['annual_inc'] <= 120000), ['annual_inc']] = 80
    df.loc[(df['annual_inc'] > 120000), ['annual_inc']] = 100
    df.loc[df['annual_inc'] == 0, ['annual_inc']] = '0~40k'
    df.loc[df['annual_inc'] == 40, ['annual_inc']] = '40~60k'
    df.loc[df['annual_inc'] == 60, ['annual_inc']] = '60~80k'
    df.loc[df['annual_inc'] == 80, ['annual_inc']] = '80~120k'
    df.loc[df['annual_inc'] == 100, ['annual_inc']] = '120k+'
    
    df.loc[(df['dti'] <= 10), ['dti']] = 0
    df.loc[(df['dti'] > 10) & (df['dti'] <= 20), ['dti']] = 1
    df.loc[(df['dti'] > 20) & (df['dti'] <= 30), ['dti']] = 2
    df.loc[(df['dti'] > 30), ['dti']] = 3
    df.loc[df['dti'] == 0, ['dti']] = 'low'
    df.loc[df['dti'] == 1, ['dti']] = 'medium'
    df.loc[df['dti'] == 2, ['dti']] = 'high'
    df.loc[df['dti'] == 3, ['dti']] = 'extreme'
    
    df.loc[(df['delinq_2yrs'] >= 2), ['delinq_2yrs']] = 2
    df.loc[df['delinq_2yrs'] == 0, ['delinq_2yrs']] = 'never'
    df.loc[df['delinq_2yrs'] == 1, ['delinq_2yrs']] = 'once'
    df.loc[df['delinq_2yrs'] == 2, ['delinq_2yrs']] = 'multiple'
    
    df.loc[(df['revol_util'] <= 30), ['revol_util']] = 0
    df.loc[(df['revol_util'] > 30) & (df['revol_util'] <= 60), ['revol_util']] = 1
    df.loc[(df['revol_util'] > 60) & (df['revol_util'] <= 100), ['revol_util']] = 2
    df.loc[(df['revol_util'] > 100), ['revol_util']] = 3
    df.loc[df['revol_util'] == 0, ['revol_util']] = 'low'
    df.loc[df['revol_util'] == 1, ['revol_util']] = 'medium'
    df.loc[df['revol_util'] == 2, ['revol_util']] = 'high'
    df.loc[df['revol_util'] == 3, ['revol_util']] = 'extreme'

In [9]:
def get_prob(df, name, parent=None):
    if parent is None:
        parent = []
    
    #Make a list of parents name
    #Get the size of each variable
    table_list = [name]
    size_dict = {name:len(df[name].value_counts())}
    row_num = len(df[name].value_counts())
    for dad in parent:
        table_list.append(dad)
        size_dict.update({dad:len(df[dad].value_counts())})
        row_num = row_num * len(df[dad].value_counts())
    
    #Make a list of list with all propogation probabilities
    name_col = np.repeat(df[name].unique(),row_num/(len(df[name].unique())))
    tb_list = [[name]]   
    for name_str in name_col:
        tb_list.append([name_str])
    counter = len(df[name].unique())
    for pt_str in parent:
        tb_list[0].append(pt_str)
        pt_col = np.tile(np.repeat(df[pt_str].unique(),row_num/(len(df[pt_str].unique())*counter)),counter)
        for i in range(0, row_num):
            tb_list[i+1].append(pt_col[i])
        counter = counter * len(df[pt_str].unique())
    
    #Add probability
    for d_row in tb_list[1:]:
        d_df = df
        for d_index in range(1,len(tb_list[0])):
            d_df = d_df[d_df[tb_list[0][d_index]] == d_row[d_index]]
        try:
            perc = d_df[name].value_counts()[d_row[0]]/len(d_df)
        except:
            perc = 0
        d_row.append(perc)
    tb_list[0].append('prob')
    table = np.array(tb_list)
    return table

In [10]:
def test_discrete(df, trained_table):
    lol = df.values.tolist()
    headers = list(df.columns.values)
    headers = np.append(headers,'prob')
    tb_list = trained_table.tolist()
    
    id_1 = df.columns.get_loc(tb_list[0][1])
    id_2 = df.columns.get_loc(tb_list[0][2])
    id_3 = df.columns.get_loc(tb_list[0][3])
    id_4 = df.columns.get_loc(tb_list[0][4])
    
    
    
    for d_row in lol:
        col_list = ['Fully Paid', d_row[id_1], d_row[id_2], d_row[id_3], d_row[id_4]]
        for t_row in tb_list[1:]:
            if col_list == t_row[:-1]:
                prob = float(t_row[-1])
                break
        d_row.append(prob)
    
    output_df = pd.DataFrame(lol, columns=headers)
    return output_df

In [11]:
test_df = copy.deepcopy(cr_data)
to_discrete(test_df)
test = get_prob(test_df[:200000],'loan_status', {'annual_inc','installment','revol_util','delinq_2yrs'})

As we can see, only a 13 entries out of 1.3 million samples have a predicted fully paid rate lower than 50%, and out of these 13 entires only 6 actually defaulted... Not good

In [12]:
discrete_results = test_discrete(test_df[200001:210000], test)
#discrete_results.head()
discrete_results.loc[discrete_results['prob'] < 0.5]

,term,installment,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,revol_util,prob
986,36 months,400~600,B,B2,MORTGAGE,0~40k,Source Verified,Fully Paid,debt_consolidation,KY,medium,multiple,medium,0.489583
2315,36 months,800+,B,B1,MORTGAGE,40~60k,Not Verified,Default,debt_consolidation,PA,high,once,medium,0.484536
3228,36 months,800+,C,C3,MORTGAGE,40~60k,Not Verified,Fully Paid,credit_card,AZ,high,once,medium,0.484536
4088,36 months,800+,D,D1,MORTGAGE,40~60k,Verified,Default,debt_consolidation,CO,high,once,medium,0.484536
4844,36 months,400~600,E,E4,MORTGAGE,0~40k,Verified,Fully Paid,debt_consolidation,OH,extreme,multiple,medium,0.489583
5118,36 months,0~200,B,B4,OWN,0~40k,Verified,Default,car,NY,low,never,extreme,0.485714
5257,36 months,800+,D,D3,MORTGAGE,40~60k,Not Verified,Fully Paid,debt_consolidation,GA,medium,once,medium,0.484536
5651,36 months,400~600,B,B2,MORTGAGE,0~40k,Source Verified,Default,credit_card,IN,medium,multiple,medium,0.489583
5752,36 months,400~600,C,C5,RENT,0~40k,Verified,Default,credit_card,TX,extreme,multiple,medium,0.489583
6482,36 months,400~600,B,B1,MORTGAGE,0~40k,Source Verified,Default,home_improvement,NY,low,multiple,medium,0.489583


## Model 2: Logistic Regression Model

In [13]:
lin_df = copy.deepcopy(cr_data)

In [14]:
def to_dummies(df):

    df_y = df['loan_status']
    df_x = df[['term','installment','home_ownership','annual_inc','verification_status','purpose','addr_state','dti','delinq_2yrs','revol_util']]
    df_x = pd.get_dummies(df_x)
    
    return [df_x, df_y]    

This function below is taken from MIE457's assignment

In [15]:
def random_log(X, y, num_tests):
    # cm_list is a list of confusion matrices for the different random splits of the dataset
    cm_list = []
    
    # Write your code here
    for num in range(num_tests):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random.randint(1,1000))
        clf = LogisticRegression(C=1.0).fit(X_train, y_train)
        y_train_predict = clf.predict(X_train)
        y_test_predict = clf.predict(X_test)
        cm_list.append(confusion_matrix(y_test, y_test_predict))
        
    # sum the confusion matrices and return the combined confusion matrix
    combined_cm = pd.Panel(cm_list).sum(axis=0)
    
    
    return combined_cm

In [16]:
[lin_df_x, lin_df_y] = to_dummies(lin_df)
cm10 =  random_log (lin_df_x, lin_df_y, num_tests = 10)
pd.DataFrame(cm10)

,0,1
0,129,863213
1,154,3127324


In [17]:
print('precision: {}'.format(cm10[1][1]/(cm10[1][0]+cm10[1][1])))
print('accuracy: {}'.format((cm10[0][0]+cm10[1][1])/(cm10[0][0]+cm10[0][1]+cm10[1][0]+cm10[1][1])))

precision: 0.7836850027953631
accuracy: 0.7836617537247984


## Model 3: Fully Connected Network

In [18]:
fcn_df = copy.deepcopy(cr_data)

In [19]:
fcn_df.head()

,term,installment,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,revol_util
0,36 months,1151.16,D,D5,MORTGAGE,100000.0,Source Verified,Fully Paid,debt_consolidation,CA,30.46,0.0,37.0
1,60 months,975.71,C,C4,MORTGAGE,45000.0,Verified,Fully Paid,credit_card,OH,50.53,0.0,64.5
2,36 months,622.68,A,A3,MORTGAGE,100000.0,Not Verified,Fully Paid,credit_card,WA,18.92,0.0,29.9
3,36 months,147.99,B,B3,RENT,38500.0,Not Verified,Fully Paid,credit_card,TX,4.64,0.0,15.3
4,36 months,345.18,E,E5,MORTGAGE,450000.0,Verified,Fully Paid,credit_card,MA,12.37,0.0,65.7


In [20]:
def to_fcn(df):
    df['loan_status']= df['loan_status'].map({'Fully Paid': 1, 'Default': 0})

    df_y = df['loan_status']
    df_x = df[['term','installment','home_ownership','annual_inc','verification_status','purpose','addr_state','dti','delinq_2yrs','revol_util']]
    df_x = pd.get_dummies(df_x)
    
    
    
    
    return [df_x, df_y]    

In [21]:
[X, y] = to_fcn(fcn_df)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random.randint(1,1000))

In [23]:
model = Sequential()
model.add(Dense(128, input_shape = [78], activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [24]:
opt = tensorflow.keras.optimizers.SGD(lr=0.001)

In [25]:
model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])

In [26]:
batch_size = 256
epochs = 10 

In [27]:
my_model = model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_test, y_test), epochs=epochs)

Train on 931189 samples, validate on 399082 samples
Epoch 1/10
931189/931189 [==============================] - 8s 9us/step - loss: 0.1799 - acc: 0.7578 - val_loss: 0.1695 - val_acc: 0.7837
Epoch 2/10
931189/931189 [==============================] - 8s 9us/step - loss: 0.1695 - acc: 0.7838 - val_loss: 0.1695 - val_acc: 0.7837
Epoch 3/10
931189/931189 [==============================] - 8s 9us/step - loss: 0.1695 - acc: 0.7838 - val_loss: 0.1695 - val_acc: 0.7837
Epoch 4/10
931189/931189 [==============================] - 8s 9us/step - loss: 0.1695 - acc: 0.7838 - val_loss: 0.1695 - val_acc: 0.7837
Epoch 5/10
931189/931189 [==============================] - 8s 9us/step - loss: 0.1695 - acc: 0.7838 - val_loss: 0.1695 - val_acc: 0.7837
Epoch 6/10
931189/931189 [==============================] - 8s 9us/step - loss: 0.1695 - acc: 0.7838 - val_loss: 0.1695 - val_acc: 0.7837
Epoch 7/10
931189/931189 [==============================] - 8s 9us/step - loss: 0.1695 - acc: 0.7838 - val_loss: 0.1695 

## Model 4: Logistic Regression + Autoencoder

In [28]:
auto_df = copy.deepcopy(cr_data)

In [29]:
auto_df['loan_status']= auto_df['loan_status'].map({'Fully Paid': 1, 'Default': 0})
categoical_vars = ['term','home_ownership','verification_status','purpose','addr_state']
preds  = auto_df['loan_status']
auto_df = auto_df.drop('loan_status',axis=1)
auto_df = auto_df.drop('grade',axis=1)
auto_df = auto_df.drop('sub_grade',axis=1)
auto_df.head()

,term,installment,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,revol_util
0,36 months,1151.16,MORTGAGE,100000.0,Source Verified,debt_consolidation,CA,30.46,0.0,37.0
1,60 months,975.71,MORTGAGE,45000.0,Verified,credit_card,OH,50.53,0.0,64.5
2,36 months,622.68,MORTGAGE,100000.0,Not Verified,credit_card,WA,18.92,0.0,29.9
3,36 months,147.99,RENT,38500.0,Not Verified,credit_card,TX,4.64,0.0,15.3
4,36 months,345.18,MORTGAGE,450000.0,Verified,credit_card,MA,12.37,0.0,65.7


In [30]:
all_cols = auto_df.columns
other_cols = [i for i in all_cols if i not in categoical_vars ]

In [31]:
def preproc(X_train ) : 

    input_list_train = []
    for c in categoical_vars :
        
        jjj = np.asarray(X_train[c].tolist())
        jjj = pd.factorize( jjj )[0]
        input_list_train.append( np.asarray(jjj)  )
        
    #the rest of the columns
    input_list_train.append(X_train[other_cols].values)
    return input_list_train
df_tr = preproc(auto_df)

The following code is modified from https://medium.com/@satnalikamayank12/on-learning-embeddings-for-categorical-data-using-keras-165ff2773fc9

In [32]:
model1 = Sequential()
model1.reset_states()
embedding_size1 = auto_df['term'].nunique()
vocab1 = embedding_size1+1
model1.add(Embedding(embedding_size1, embedding_size1, input_length = 1))
model1.add(Reshape(target_shape=(embedding_size1,)))

model2 = Sequential()
model2.reset_states()
embedding_size2 = auto_df['home_ownership'].nunique()
vocab2 = embedding_size2+1
model2.add(Embedding(embedding_size2, embedding_size2, input_length = 1 ))
model2.add(Reshape(target_shape=(embedding_size2,)))

model3 = Sequential()
model3.reset_states()
embedding_size3 = auto_df['verification_status'].nunique()
vocab3 = embedding_size3+1
model3.add(Embedding(embedding_size3, embedding_size3, input_length = 1 ))
model3.add(Reshape(target_shape=(embedding_size3,)))

model4 = Sequential()
model4.reset_states()
embedding_size4 = auto_df['purpose'].nunique()
vocab4  = embedding_size4+1
model4.add(Embedding(embedding_size4, embedding_size4, input_length = 1 ))
model4.add(Reshape(target_shape=(embedding_size4,)))

model5 = Sequential()
model5.reset_states()
embedding_size5 = auto_df['addr_state'].nunique()
vocab5  = embedding_size5+1
model5.add(Embedding(embedding_size5, embedding_size5, input_length = 1 ))
model5.add(Reshape(target_shape=(embedding_size5,)))

model_rest = Sequential()
model_rest.add(Dense(64 , input_dim = 5))
model_rest.reset_states( )

com_model = concatenate([model1.output, model2.output, model3.output, model4.output, model5.output, model_rest.output])

In [33]:
full_model = Sequential()

full_model.add(Dense(1024))
full_model.add(Activation('relu'))
full_model.add(Dropout(0.2))

full_model.add(Dense(256))
full_model.add(Activation('relu'))
full_model.add(Dropout(0.2))

full_model.add(Dense(1))

In [34]:
final_model = Model([model1.input, model2.input, model3.input, model4.input, model5.input, model_rest.input], full_model(com_model))
final_model.compile(loss='binary_crossentropy', optimizer='Adam')

In [35]:
history = final_model.fit(df_tr, preds, epochs =  10, batch_size = 128)

C:\Users\Zheng\Anaconda3\envs\2019_tf2_env\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
1330271/1330271 [==============================] - 87s 65us/step - loss: 8.4559
Epoch 2/10
1330271/1330271 [==============================] - 87s 66us/step - loss: 3.4473
Epoch 3/10
1330271/1330271 [==============================] - 86s 65us/step - loss: 3.4473
Epoch 4/10
1330271/1330271 [==============================] - 86s 65us/step - loss: 3.4473
Epoch 5/10
1330271/1330271 [==============================] - 86s 65us/step - loss: 3.4473
Epoch 6/10
1330271/1330271 [==============================] - 87s 65us/step - loss: 3.4473
Epoch 7/10
1330271/1330271 [==============================] - 88s 66us/step - loss: 3.4473
Epoch 8/10
1330271/1330271 [==============================] - 87s 65us/step - loss: 3.4473
Epoch 9/10
1330271/1330271 [==============================] - 87s 65us/step - loss: 3.4473
Epoch 10/10
1330271/1330271 [==============================] - 86s 65us/step - loss: 3.4473


In [36]:
w1 = final_model.layers[5].get_weights()
w2 = final_model.layers[6].get_weights()
w3 = final_model.layers[7].get_weights()
w4 = final_model.layers[8].get_weights()
w5 = final_model.layers[9].get_weights()
w = [w1,w2,w3,w4,w5]

In [37]:
def embed_df(df, df_tr, w):
    df.drop('term', axis = 1, inplace = True)
    df.drop('home_ownership', axis = 1, inplace = True)
    df.drop('verification_status', axis = 1, inplace = True)
    df.drop('purpose', axis = 1, inplace = True)
    df.drop('addr_state', axis = 1, inplace = True)
    
    category_name = ['term','home_ownership','verification_status','purpose','addr_state']
    
    for name in category_name:
        n_index = category_name.index(name)
        for i in range(0, len(w[n_index][0])):
            col_name = '{} {}'.format(name, i)
            df[col_name] = df_tr[n_index]
            n_dict = {}
            for j in range(0, len(w[n_index][0])):
                n_dict.update({j:w[n_index][0][j][i]})
            df[col_name] = df[col_name].map(n_dict)
    
    return df

In [38]:
test_df = copy.deepcopy(cr_data)

In [39]:
test_df = embed_df(test_df, df_tr,w)

In [40]:
def to_test(df):

    df_y = df['loan_status']
    df_x = df.drop(['loan_status','grade','sub_grade'], axis=1)
    
    
    return [df_x, df_y]    

In [41]:
[t_df_x, t_df_y] = to_test(test_df)

In [42]:
cm2 =  random_log (t_df_x, t_df_y, num_tests = 10)
pd.DataFrame(cm2)

,0,1
0,128,862942
1,149,3127601


## Additional Analysis

In [43]:
def analyze_data(df):
    grade_list = set(df['grade'].tolist())
    sub_grade_list = set(df['sub_grade'].tolist())
    
    grade_name = []
    grade_perc = []
    sub_grade_name = []
    sub_grade_perc = []
    
    for this_grade in grade_list:
        grade_df = df[df.grade == this_grade]
        if len(grade_df['loan_status'].value_counts()) == 2:
            perc = grade_df['loan_status'].value_counts()[0]/(grade_df['loan_status'].value_counts()[0]+grade_df['loan_status'].value_counts()[1])
        elif grade_df['loan_status'].value_counts().keys()[0] == 'Default':
            perc = 0
        else:
            perc = 1
        grade_perc.append(perc)
        grade_name.append(this_grade)
    
    for that_grade in sub_grade_list:
        sub_grade_df = df[df.sub_grade == that_grade]
        if len(sub_grade_df['loan_status'].value_counts()) == 2:
            sub_perc = sub_grade_df['loan_status'].value_counts()[0]/(sub_grade_df['loan_status'].value_counts()[0]+sub_grade_df['loan_status'].value_counts()[1])
        elif sub_grade_df['loan_status'].value_counts().keys()[0] == 'Default':
            sub_perc = 0
        else:
            sub_perc = 1
        sub_grade_perc.append(sub_perc)
        sub_grade_name.append(that_grade)
    
    this_d = {'grade':grade_name,'payback_rate':grade_perc}
    this_df = pd.DataFrame(data=this_d)
    this_df = this_df.set_index('grade').sort_index()
    print(this_df)
    
    that_d = {'sub_grade':sub_grade_name,'payback_rate':sub_grade_perc}
    that_df = pd.DataFrame(data=that_d)
    that_df = that_df.set_index('sub_grade').sort_index()
    print(that_df)

In [44]:
analyze_data(cr_data)

       payback_rate
grade              
A          0.932625
B          0.853035
C          0.756580
D          0.675571
E          0.597655
F          0.533508
G          0.513876
           payback_rate
sub_grade              
A1             0.963330
A2             0.947411
A3             0.937556
A4             0.923349
A5             0.908449
B1             0.884263
B2             0.875420
B3             0.858656
B4             0.837858
B5             0.815937
C1             0.794084
C2             0.775330
C3             0.756062
C4             0.729700
C5             0.717887
D1             0.702889
D2             0.683880
D3             0.672963
D4             0.655619
D5             0.643881
E1             0.623495
E2             0.609566
E3             0.593987
E4             0.581476
E5             0.560395
F1             0.561905
F2             0.533892
F3             0.535211
F4             0.506362
F5             0.507589
G1             0.502628
G2             0.501606
G3  